In [6]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [7]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("Optimize Efficacy Tools")
mlflow.autolog()
mlflow.tracing.disable_notebook_display()

load_dotenv("../.env")
dspy.configure(
    lm=dspy.LM("gemini/gemini-2.5-pro", temperature=0.5, cache=True, rollout_id=1)
)

COMPOUND = "Luminespib"

2025/09/15 13:43:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/09/15 13:43:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/09/15 13:43:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [8]:
efficacy_agent = CFEfficacyAgent(max_iters=12)
efficacy_agent.agent.react.signature = efficacy_agent.agent.react.signature.insert(
    0,
    "tool_expected_info",
    dspy.OutputField(
        desc="TRUE or FALSE: <more info>. If FALSE, describe what information (ex databases, literature, etc) was desired from tool. Do not say FALSE if tool did not return any information (ex due to no information available), only say FALSE if tool returned information that was not useful or other information was desired from this tool call.\n\n"
    ),
    type_=str,
)
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

Prediction(
    trajectory={'thought_0': "The first step is to identify the compound 'Luminespib' in the ChEMBL database to retrieve its unique identifier. This ID will be crucial for subsequent queries to gather information on its bioactivity, properties, and potential relevance to cardiac fibrosis.", 'tool_name_0': 'search_chembl_id', 'tool_args_0': {'query': 'Luminespib'}, 'observation_0': "Found 2 compound(s) matching 'Luminespib': \n - CHEMBL3137334 (LUMINESPIB MESYLATE)\n - CHEMBL252164 (LUMINESPIB)", 'thought_1': 'Now that I have the ChEMBL ID for Luminespib (CHEMBL252164), the next step is to investigate its biological activity. Understanding its primary targets and potency is crucial for forming a hypothesis about its potential efficacy in cardiac fibrosis. I will use the `get_compound_bioactivities_summary` tool to get an overview of its known biological effects.', 'tool_name_1': 'get_compound_bioactivities_summary', 'tool_args_1': {'chembl_id': 'CHEMBL252164'}, 'observation_

In [ ]:
# tox_agent = ToxicityScreeningAgent(max_iters=12)
# tox_results = tox_agent(compound_name=COMPOUND)
# print(tox_results)

In [ ]:
# prioritization_agent = CompoundPrioritizationAgent()
# prioritization_results = prioritization_agent(compound_name=COMPOUND)
# print(prioritization_results)